# ДЗ к уроку №8. Снижение размерности данных 

## 1. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection
import numpy as np

In [2]:
from sklearn.datasets import load_iris

In [3]:
X, y = load_iris(return_X_y=True)

### PCA

Для закрепления теории реализуем PCA с помощью Python.

In [4]:
# Загрузим игрушечный датасет из sklearn
iris = load_iris()
X = iris.data
X.shape

(150, 4)

In [5]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [6]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i[0])

Собственные значения в порядке убывания:
437.7746724797988
137.10457072021055
22.013531335697195
3.107225464292886


Оценим долю дисперсии, которая описывается найденными компонентами.

In [7]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[72.96244541329987, 22.850761786701778, 3.6689218892828697, 0.5178709107154814]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. В то же время последняя компонента описывает всего 0.5% и может быть отброжена без страха значительных потерь в качестве нашего анализа. Мы отбросим последние две компоненты, оставив первые две.

In [8]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [9]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_.dot(W)

In [10]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [11]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = model_selection.train_test_split(Z, y, test_size=0.25)

### Стохастический градиентный бустинг

In [12]:
def gb_predict(X, trees_list, coef_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
    return np.array([sum([eta* coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

In [13]:
def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

In [14]:
def bias(y, z):
    return (y - z)

In [15]:
def stochastic_gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    trees = []
    train_errors = []
    test_errors = []
    
    np.random.seed(42)
    n_samples = X_train.shape[0]
    n_samples_for_sgb = int(0.5 * n_samples)
    idx = np.arange(n_samples)
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
        
        ind = np.random.choice(idx, n_samples_for_sgb)

        if len(trees) == 0:
            tree.fit(X_train[ind], y_train[ind])            
        else:
            target = gb_predict(X_train[ind], trees, coefs, eta)
            tree.fit(X_train[ind], bias(y_train[ind], target))
            
        train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
        test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))

        trees.append(tree)
        
    return trees, train_errors, test_errors

In [16]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta):
    train_prediction = gb_predict(X_train, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тренировочной выборке: {mean_squared_error(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тестовой выборке: {mean_squared_error(y_test, test_prediction)}')

### Сравнение

In [17]:
# Число деревьев в ансамбле
n_trees = 10

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 15

# Шаг
eta = 1

trees, train_errors, test_errors = stochastic_gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)
trees_pca, train_errors_pca, test_errors_pca = stochastic_gb_fit(n_trees, max_depth, X_train_pca, X_test_pca, y_train_pca, y_test_pca, coefs, eta)

In [18]:
print('\033[1m'+'ДО применения PCA:'+'\033[0m')
evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)
print('\n'+'\033[1m'+'ПОСЛЕ применения PCA:'+'\033[0m')
evaluate_alg(X_train_pca, X_test_pca, y_train_pca, y_test_pca, trees_pca, coefs, eta)

ДО применения PCA:
Ошибка алгоритма из 10 деревьев глубиной 15     с шагом 1 на тренировочной выборке: 0.017857142857142856
Ошибка алгоритма из 10 деревьев глубиной 15     с шагом 1 на тестовой выборке: 0.18421052631578946

ПОСЛЕ применения PCA:
Ошибка алгоритма из 10 деревьев глубиной 15     с шагом 1 на тренировочной выборке: 0.05357142857142857
Ошибка алгоритма из 10 деревьев глубиной 15     с шагом 1 на тестовой выборке: 0.21052631578947367


**Вывод:** с понижением размерности качество немного уменьшилось.